In [1]:
import os
from langchain_core.tools import tool, StructuredTool
import asyncio
import pandas as pd
import tiktoken
from langchain.agents import Tool
import asyncio
from typing import Any, Dict, List
from graph_search import setup_global_search, setup_local_search, create_search_engine
from langchain.callbacks.base import AsyncCallbackHandler, BaseCallbackHandler
from langchain_core.outputs import LLMResult
from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import GlobalCommunityContext
from graphrag.query.structured_search.global_search.search import GlobalSearch
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI as OAI

#Define api_key and llm_model
api_key = #PLACEHOLDER
llm_model = 'gpt-3.5-turbo'


llm = ChatOpenAI(
        api_key=api_key,
        model=llm_model,
        api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
        max_retries=20,
    )

#Formats response to input into LLM chain
def format_response(query, graph_rag_answer):
    prompt = f"Given the following question by the user and the database result synthesize an answer:\n\n"
    prompt += f"User Question: {query}"
    prompt += f"Database Answer: {graph_rag_answer}"
    return prompt

#Queries the graph
async def query_graph(prompt, clearance_level = 'general'):

    input_dir = #Define input dir 
    #based on clearance_level parameter query the correct file (CHANGE THIS FOR YOUR OWN INDEXES)
    #clearance_dir example clearance_lev/output/20287712-105687/artifacts
    if clearance_level == 'admin':
        clearance_dir = #place holder
    
    elif clearance_level == 'level_q':
        clearance_dir = #placeholder

    elif clearance_level == 'general':
        clearance_dir = #placeholder

        

    token_encoder = tiktoken.get_encoding("cl100k_base")
    
    # parquet files generated from indexing pipeline
    INPUT_DIR = f"{input_dir}/{clearance_dir}"
    COMMUNITY_REPORT_TABLE = "create_final_community_reports"
    ENTITY_TABLE = "create_final_nodes"
    ENTITY_EMBEDDING_TABLE = "create_final_entities"


    # community level in the Leiden community hierarchy from which we will load the community reports
    # higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
    COMMUNITY_LEVEL = 2

    entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
    report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
    entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

    reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
    entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
    report_df.head()

    context_builder = GlobalCommunityContext(
        community_reports=reports,
        entities=entities,  # default to None if you don't want to use community weights for ranking
        token_encoder=token_encoder,
    )
    context_builder_params = {
        "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
        "shuffle_data": True,
        "include_community_rank": True,
        "min_community_rank": 0,
        "community_rank_name": "rank",
        "include_community_weight": True,
        "community_weight_name": "occurrence weight",
        "normalize_community_weight": True,
        "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
        "context_name": "Reports",
    }

    map_llm_params = {
        "max_tokens": 1000,
        "temperature": 0.0,
        "response_format": {"type": "json_object"},
    }

    reduce_llm_params = {
        "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
        "temperature": 0.0,
    }
    search_engine = GlobalSearch(
        llm=llm,
        context_builder=context_builder,
        token_encoder=token_encoder,
        max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
        map_llm_params=map_llm_params,
        reduce_llm_params=reduce_llm_params,
        allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
        json_mode=True,  # set this to False if your LLM model does not support JSON mode.
        context_builder_params=context_builder_params,
        concurrent_coroutines=32,
        response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
    )

    result = await search_engine.asearch(prompt)
    return  result.response




/Users/lil_lawnmower/Desktop/Code/graphrag/test_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
query = 'where is the main character walking to?'
graph_rag_answer = await query_graph(prompt = 'where is the main character walking to?', clearance_level = 'level_q')

In [ ]:
formatted_prompt = format_response(query, graph_rag_answer)

#Define LLM for the langchain
llm = OAI(
        api_key=api_key,
        model=llm_model,
    )

#Define prompt to give LLM chain
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that synthesizes information and give relevant information back to the user. You are also honest if you don't know the answer to a question. If the question doesn't need specific data and can be answered with general knowledge, you can answer it. If the question needs specific data, you can ask for clarification. If the question is too broad, you can ask for more details. If the question is inappropriate, you can refuse to answer.",
        ),
        ("human", "{input}"),
    ]
)

#Chain LLM and prompt
chain = prompt | llm

#Call LLM
answer = chain.invoke({
    "input": formatted_prompt
})

print(answer.content)


Based on the information provided in the dataset, the main character is depicted walking from Shin Kobe Station to Sannomiya in Kobe. This journey is not just a physical transit but holds deeper significance within the narrative. It symbolizes emotional connections, transitions, and pivotal moments in the character's development. The repeated emphasis on this specific route suggests that it plays a crucial role in shaping the character's experiences and the overall storyline. Further exploration into how this journey influences the character's development and the broader themes of the narrative could provide valuable insights into the narrative's complexities and character dynamics.
